In [1]:
from SimPEG import maps as Maps
#from SimPEG import utils as Utils
#from SimPEG import simulation as Problem
#from SimPEG import props as Props
# from SimPEG import models as Models
# from SimPEG import survey as Survey
# from SimPEG import regularization as Regularization
# from SimPEG import data_misfit as DataMisfit
# from SimPEG import inverse_problem as InvProblem
# from SimPEG import optimization as Optimization
# from SimPEG import directives as Directives
# from SimPEG import inversion as Inversion
from simpegEM1D import EM1D, EM1DSurveyTD, Utils1D, get_vertical_discretization_time, set_mesh_1d, piecewise_pulse
import numpy as np

In [6]:
from simpegEM1D import skytem_HM_2015, skytem_LM_2015
wave_HM = skytem_HM_2015()
wave_LM = skytem_LM_2015()
time_HM = wave_HM.time_gate_center
time_LM = wave_LM.time_gate_center
def get_problem_survey(hz):
    mesh1D = set_mesh_1d(hz)
    depth = -mesh1D.gridN[:-1]
    LocSigZ = -mesh1D.gridCC
    expmap = Maps.ExpMap(mesh1D)    
    
    time_input_currents_HM = wave_HM.current_times[-7:]
    input_currents_HM = wave_HM.currents[-7:]
    time_input_currents_LM = wave_LM.current_times[-13:]
    input_currents_LM = wave_LM.currents[-13:]

    TDsurvey = EM1DSurveyTD(
        rx_location = np.array([0., 0., 100.+30.]),
        src_location = np.array([0., 0., 100.+30.]),
        topo = np.r_[0., 0., 100.],
        depth = depth,
        rx_type = 'dBzdt',
        wave_type = 'general',
        src_type = 'CircularLoop',
        a = 13.,
        I = 1.,
        time = time_HM,
        time_input_currents=time_input_currents_HM,
        input_currents=input_currents_HM,
        n_pulse = 2,
        base_frequency = 25.,
        use_lowpass_filter=True,
        high_cut_frequency=210*1e3,
        moment_type='dual',
        time_dual_moment = time_HM,    
        time_input_currents_dual_moment=time_input_currents_LM,
        input_currents_dual_moment=input_currents_LM,
        base_frequency_dual_moment=210,    
    )
    
    prob = EM1D(mesh1D, sigmaMap=expmap, verbose=False)
#    if prob.ispaired:
#        prob.unpair()
#    if TDsurvey.ispaired:
#        TDsurvey.unpair()
#    prob.pair(TDsurvey)
    prob.survey = TDsurvey
#    TDsurvey.pair(prob)
    prob.chi = np.zeros(TDsurvey.n_layer)
    return TDsurvey, prob

In [7]:
hz = np.r_[100, 30, 180, 1000.]
sig = np.r_[1./50, 1./5, 1./30, 1./1.]
mesh1D = set_mesh_1d(hz)
depth = -mesh1D.gridN[:-1]
LocSigZ = -mesh1D.gridCC

In [8]:
hz

array([ 100.,   30.,  180., 1000.])

In [10]:
TDsurvey, prob = get_problem_survey(hz)
sig_half = 1./50.
chi_half = 0.
m_true = np.log(sig)
std = 0.05
TDsurvey.pair(prob)
dtrue = TDsurvey.dpred(m_true)
np.random.seed(1)
noise = std*abs(dtrue)*np.random.randn(*dtrue.shape)
dobs = dtrue+noise

{'sigmaMap': <SimPEG.maps.ExpMap object at 0x7f3fd0a0c160>, 'verbose': False}


AttributeError: 'EM1DSurveyTD' object has no attribute 'pair'

In [ ]:
import matplotlib
matplotlib.rcParams['font.size'] = 14

In [ ]:
fig, ax = subplots(1,1, figsize=(4, 5))
Utils1D.plotLayer(1./sig, mesh1D, showlayers=False, ax=ax)
ax.set_xlabel("Resistivity ($\Omega$m)")
plt.tight_layout()
fig.savefig('resistivity', dpi=200)
ax.set_xlim(0.5, 100)
ax.set_ylim(-600, 0)

In [ ]:
fig, axes = subplots(1,1, figsize = (6,4))
axes.plot(TDsurvey.time, -dobs[:TDsurvey.n_time], '-', lw=2)
axes.plot(TDsurvey.time_dual_moment, -dobs[TDsurvey.n_time:], '-', lw=2)
axes.set_xscale('log');
axes.set_yscale('log');
plt.legend(("HM", "LM"))
plt.xlabel("Time (s)")
plt.ylabel("SkyTEM data (V/A-m$^2$)")
plt.tight_layout()
fig.savefig('skytem_data', dpi=200)

In [ ]:
# std = 0.05
# prob.getJ(m_true)
# prob.depth_of_investigation_christiansen_2012(std, thres_hold=0.8)

In [ ]:
IRLS = Directives.Update_IRLS(
        maxIRLSiter=40, minGNiter=1, fix_Jmatrix=True, coolingRate=1, coolingFactor=2, betaSearch=False,
        chifact_start = 1.
    )

In [ ]:
def run_inversion(m0, dobs, std, floor, is_sparse=True):
    hz = get_vertical_discretization_time(
        np.unique(np.r_[time_HM, time_LM]), facter_tmax=0.5, factor_tmin=10., n_layer=30
    )    
    mesh1D = set_mesh_1d(hz)
    TDsurvey, prob = get_problem_survey(hz)
    TDsurvey.dobs = dobs.copy()
    uncert = abs(TDsurvey.dobs)*std+floor
    dmisfit = DataMisfit.l2_DataMisfit(TDsurvey)
    dmisfit.W = 1./ uncert
    
    
    if is_sparse:
        reg = Regularization.Sparse(
            mesh1D,
            mapping=Maps.IdentityMap(mesh1D),
            alpha_s=1.,
            alpha_x=1.,
        )
        p = 0
        qx, qz = 1., 1.
        reg.norms = np.c_[p, qx, qz, 0.]
        IRLS = Directives.Update_IRLS(
            maxIRLSiter=40, minGNiter=1, fix_Jmatrix=True, coolingRate=1, coolingFactor=5, betaSearch=False,
            chifact_start = 1.,
            f_min_change = 1e-3
        )        
    else:
        reg = Regularization.Tikhonov(
            mesh1D,
            mapping=Maps.IdentityMap(mesh1D),
            alpha_s=1.,
            alpha_x=0.05,
        )        
    opt = Optimization.ProjectedGNCG(maxIter = 40)
    invProb = InvProblem.BaseInvProblem(dmisfit, reg, opt)
    beta = Directives.BetaSchedule(coolingFactor=2., coolingRate=1)
    betaest = Directives.BetaEstimate_ByEig(beta0_ratio=10**1.5)
    target = Directives.TargetMisfit()
    save = Directives.SaveOutputEveryIteration()
    if is_sparse:
        directiveList=[IRLS, betaest]
    else:
        directiveList=[beta, betaest, target]
    
    inv = Inversion.BaseInversion(invProb, directiveList=directiveList)
    prob.counter = opt.counter = Utils.Counter()
    opt.LSshorten = 0.5
    opt.remember('xc')
    mopt = inv.run(m0)
    return mopt, invProb

In [ ]:
%%time
m0 = np.log(np.ones(30)*1./50)
mopt, invProb = run_inversion(m0, dobs, 0.1, 0., is_sparse=False)

In [ ]:
m0_1 = np.log(np.ones(30)*1./30)
mopt_1, invProb_1 = run_inversion(m0_1, dobs, 0.1, 0., is_sparse=False)

In [ ]:
def compute_doi_index_oldenburg_and_li_1999(m1, m2, m1r, m2r):
    r = abs((m1-m2) / (m1r-m2r))
    r[r>0.99] = 1.
    return r

In [ ]:
# fig, ax = subplots(1,1, figsize=(4, 5))
# Utils1D.plotLayer(1./sig, mesh1D, showlayers=False, ax=ax)
# Utils1D.plotLayer(1./(expmap*invProb.l2model), mesh1D, showlayers=False, **{'color':'r', 'lw':3})
# ax.set_xlabel("Resistivity ($\Omega$m)")
# ax.set_xlim(0.5, 300)
# plt.tight_layout()
# fig.savefig('resistivity_l2', dpi=200)

In [ ]:
mesh1D_inv = invProb.dmisfit.prob.mesh

In [ ]:
# fig, ax = subplots(1,1, figsize=(4, 5))
# Utils1D.plotLayer(1./sig, mesh1D, showlayers=False, ax=ax)
# Utils1D.plotLayer(1./(np.log(mopt)), mesh1D_inv, showlayers=False, **{'color':'r', 'lw':3})
# ax.set_xlabel("Resistivity ($\Omega$m)")
# ax.set_xlim(0.5, 300)
# ax.set_ylim(-600, 0)
# plt.tight_layout()
# fig.savefig('resistivity_l2', dpi=200)


In [ ]:
fig, ax = subplots(1,1, figsize=(4, 5))
Utils1D.plotLayer(1./(np.exp(mopt_1)), mesh1D_inv, showlayers=False, **{'color':'b', 'lw':3})
Utils1D.plotLayer(1./(np.exp(mopt)), mesh1D_inv, showlayers=False, **{'color':'r', 'lw':3})
Utils1D.plotLayer(1./sig, mesh1D, showlayers=False, ax=ax)
ax.set_xlabel("Resistivity ($\Omega$m)")
ax.set_xlim(0.5, 300)
ax.set_ylim(-600, 0)
plt.tight_layout()
ax.legend(("$m_{ref}$=30 $\Omega$m", "$m_{ref}$=50 $\Omega$m"), bbox_to_anchor=(1,1))

fig.savefig('resistivity_l2_compare', dpi=200)


In [ ]:
m0 = np.log(np.ones(30)*1./50)
mopt_sparse, invProb_sparse = run_inversion(m0, dobs, 0.1, 0., is_sparse=True)

In [ ]:
m0_1 = np.log(np.ones(30)*1./30)
mopt_sparse_1, invProb_sparse_1 = run_inversion(m0_1, dobs, 0.1, 0., is_sparse=True)

In [ ]:
# fig, ax = subplots(1,1, figsize=(4, 5))
# Utils1D.plotLayer(1./sig, mesh1D, showlayers=False, ax=ax)
# Utils1D.plotLayer(1./(expmap*mopt_sparse), mesh1D_inv, showlayers=False, **{'color':'r', 'lw':3})
# ax.set_xlabel("Resistivity ($\Omega$m)")
# ax.set_xlim(0.5, 300)
# ax.set_ylim(-600, 0)
# plt.tight_layout()
# fig.savefig('resistivity_l0', dpi=200)


In [ ]:
fig, ax = subplots(1,1, figsize=(4, 5))
Utils1D.plotLayer(1./sig, mesh1D, showlayers=False, ax=ax)
Utils1D.plotLayer(1./(np.exp(mopt_sparse_1)), mesh1D_inv, showlayers=False, **{'color':'r', 'lw':3})
ax.set_xlabel("Resistivity ($\Omega$m)")
ax.set_xlim(0.5, 300)
ax.set_ylim(-600, 0)
plt.tight_layout()
fig.savefig('resistivity_l0', dpi=200)


In [ ]:
fig, ax = subplots(1,1, figsize=(4, 5))
Utils1D.plotLayer(1./(np.exp(mopt_sparse_1)), mesh1D_inv, showlayers=False, **{'color':'b', 'lw':3})
Utils1D.plotLayer(1./(np.exp(mopt_sparse)), mesh1D_inv, showlayers=False, **{'color':'r', 'lw':3})
# ax.legend(("$m_{ref}$=30 $\Omega$m", "$m_{ref}$=50 $\Omega$m"), bbox_to_anchor=(1,1))
Utils1D.plotLayer(1./sig, mesh1D, showlayers=False, ax=ax)
ax.set_xlabel("Resistivity ($\Omega$m)")
ax.set_xlim(0.5, 300)
ax.set_ylim(-600, 0)
plt.tight_layout()
fig.savefig('resistivity_l0_compare', dpi=200)


In [ ]:
fig, ax = subplots(1,1, figsize=(4, 5))
doi = compute_doi_index_oldenburg_and_li_1999(mopt, mopt_1, m0, m0_1)
Utils1D.plotLayer(doi, mesh1D_inv, showlayers=False, xscale='linear', ax=ax)
# Utils1D.plotLayer(doi_sparse, mesh1D_inv, showlayers=False, xscale='linear', **{'linestyle':'--', 'color':'k'})
ax.set_xticks(np.arange(20)*0.1, minor=True)
ax.set_xlabel("DOI index")
# ax.legend(("L2", "L0"))
ax.set_ylim(-600, 0)
plt.tight_layout()
fig.savefig('doi_index', dpi=200)

In [ ]:
majorLocator = MultipleLocator(20)
majorFormatter = FormatStrFormatter('%d')
minorLocator = MultipleLocator(5)
ax.xaxis.set_major_locator(majorLocator)
ax.xaxis.set_major_formatter(majorFormatter)
# for the minor ticks, use no labels; default NullFormatter
ax.xaxis.set_minor_locator(minorLocator)

In [ ]:
doi_sparse = compute_doi_index_oldenburg_and_li_1999(mopt_sparse, mopt_sparse_1, m0, m0_1)

In [ ]:
doi_sparse

In [ ]:
fig, axes = subplots(1,1, figsize = (7,5))
axes.plot(TDsurvey.time, -invProb.dpred[:TDsurvey.n_time], 'x')
axes.plot(TDsurvey.time_dual_moment, -invProb.dpred[TDsurvey.n_time:], 'x')
axes.plot(TDsurvey.time, -dobs[:TDsurvey.n_time])
axes.plot(TDsurvey.time_dual_moment, -dobs[TDsurvey.n_time:])

axes.set_xscale('log');
axes.set_yscale('log');